# Setting the environment


In [1]:
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'lightweight-human-pose-estimation.pytorch'

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/lightweight-human-pose-estimation.pytorch


In [ ]:
!pip install -r requirements.txt
!pip install onnx

# Downloading openvino
This will take around 15 mins to download

In [ ]:
!pip install openvino-colab
!pip install openvino
import openvino_colab #choose 5 generally, this step generally takes some time
install_dir = "/opt/intel/openvino_2021/"
model_optimizer = '/opt/intel/openvino_2021/deployment_tools/model_optimizer/'
deployment_tools = '/opt/intel/openvino_2021/deployment_tools/'
model_zoo = '/opt/intel/openvino_2021/deployment_tools/open_model_zoo/'
!python $model_zoo'tools/downloader/'downloader.py --name vehicle-attributes-recognition-barrier-0039 --precisions FP32 -o /content/openvino-colab/demo_files/models

In [ ]:
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore
import warnings
from google.colab.patches import cv2_imshow
warnings.filterwarnings("ignore", category=DeprecationWarning)

def load_IR_to_IE(model_xml):
    ### Load the Inference Engine API
    plugin = IECore()
    ### Loading the IR files to IENetwork class
    model_bin = model_xml[:-3]+"bin"
    network = IENetwork(model=model_xml, weights=model_bin)
    ### Loading the network
    executable_net = plugin.load_network(network,"CPU")
    print("Network succesfully loaded into the Inference Engine")
    return executable_net

def synchronous_inference(executable_net, image):
    ### Get the input blob for the inference request
    input_blob = next(iter(executable_net.inputs))
    ### Perform Synchronous Inference
    result = executable_net.infer(inputs = {input_blob: image})
    return result

# Converting into onnx file


In [ ]:
!python scripts/convert_to_onnx.py --checkpoint-path checkpoint_iter_370000.pth

# Using vino to optimize to real time

In [ ]:
!python $model_optimizer''/mo.py --input_model human-pose-estimation.onnx

# Testing the model on an image

In [ ]:
!python demo.py --checkpoint-path checkpoint_iter_370000.pth --image '/content/drive/MyDrive/lightweight-human-pose-estimation.pytorch/test.jpg'

# Testing the model on a video

In [40]:
!python demo.py --checkpoint-path checkpoint_iter_370000.pth --video '/content/drive/MyDrive/lightweight-human-pose-estimation.pytorch/test.mp4'

# Saving the video

In [41]:
import cv2
import os

# Path to the folder containing images
image_folder = 'test'

# Output video file name and properties
output_video_path = 'rishi.mp4'
fps = 30.0  # Frames per second

# Get the list of image files in the folder
image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.png', '.jpg', '.jpeg'))]

# Read the dimensions of the first image to set up the VideoWriter
first_image = cv2.imread(image_files[0])
height, width, layers = first_image.shape

# Create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can try other codecs as well
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Iterate through the list of images and write each frame to the video
for image_file in image_files:
    img = cv2.imread(image_file)
    out.write(img)

# Release the VideoWriter object
out.release()

print(f"Video saved to {output_video_path}")


Video saved to rishi.mp4
